In [6]:
import astropy.table as tb
from astropy.time import Time


In [48]:
dssi_sessions = tb.Table.read("../2024-07-08 Process DSSI olist logs/DSSI sessions.csv")

tic_sessions = dssi_sessions[(dssi_sessions["TIC ID"].mask == False) & (dssi_sessions["MidTime JD"] > 0)]
tic_sessions["MidTime Byear"] = Time(tic_sessions["MidTime JD"], format="jd").byear
tic_sessions#[tic_sessions["Target Name"] == "TIC 269811101"]


Target Name,TIC ID,Speckle Session,StartTime JD,MidTime JD,EndTime JD,MidTime UTC,Num Sequences,MidTime Byear
str12,str13,int32,float64,float64,float64,str19,int32,float64
H300038,TIC 328181241,12,2459849.8666666667,2459849.8680555555,2459849.8694444443,2022-09-27 08:50:00,3,2022.739252707312
H300043,TIC 168789840,20,2459849.929861111,2459849.93125,2459849.9326388887,2022-09-27 10:21:00,3,2022.739425727967
H300045,TIC 454140642,22,2459849.9381944444,2459849.939236111,2459849.9402777776,2022-09-27 10:32:30,3,2022.7394475932147
H300046,TIC 139650665,29,2459849.961111111,2459849.963194445,2459849.965277778,2022-09-27 11:07:00,3,2022.7395131889587
H300047,TIC 178953404,30,2459849.967361111,2459849.9697916666,2459849.972222222,2022-09-27 11:16:30,5,2022.7395312515532
H300051,TIC 200094011,39,2459850.019444444,2459850.020833333,2459850.0222222223,2022-09-27 12:30:00,3,2022.7396709990046
H300036,TIC 389836747,43,2459850.834027778,2459850.834027778,2459850.834027778,2022-09-28 08:01:00,1,2022.7418974516095
H300032,TIC 283940788,46,2459850.8486111113,2459850.8506944445,2459850.8527777777,2022-09-28 08:25:00,5,2022.7419430834304
H300033,TIC 284814380,48,2459850.8583333334,2459850.8604166666,2459850.8625,2022-09-28 08:39:00,5,2022.7419697019925


In [8]:
ephemerides = tb.vstack([
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2022 Binary Parameters.csv"),
    tb.Table.read("../../django/TargetDB/Data Files/Binary Parameters/Kostov 2023 Binary Parameters.csv"),
])
ephemerides

Local ID,Member,Period,T0 Primary,T0 Secondary,Depth Primary,Depth Secondary,Duration Primary,Duration Secondary
str14,str1,float64,float64,float64,int32,int32,float64,float64
TIC 9493888,A,2.098992,1816.2345,1817.283786,146,117,2.9,2.7
TIC 25818450,A,10.132402,1769.9109,1776.391584,12,9,--,--
TIC 27543409,A,2.122862,1493.1001,1494.153889,50,15,--,--
TIC 31928452,A,2.8823,1337.9129,1339.358085,30,25,2.1,1.6
TIC 52856877,A,5.186818,1791.059,1793.652928,220,90,5.5,5.5
TIC 63459761,A,4.244072,1715.1118,1717.168477,15,10,7.0,9.2
TIC 75740921,A,0.93308,1519.6949,1520.16256,87,59,2.7,2.6
TIC 78568780,A,2.88838,1468.5374,1469.962816,57,20,3.9,3.7
TIC 79140936,A,3.54389,1468.3795,1469.78607,23,7,3.7,3.4


In [110]:
eclipses = tb.Table(names=["TIC ID", "Speckle Session", "Member", "Component", "Eclipse Beg JD", "Eclipse End JD", "Session Beg JD", "Session Mid JD", "Session End JD", "Session Mid UTC", "Session Mid Byear", "T0", "Period", "Half Duration Days", "Depth", "Cycles", "Eclipse Fraction", "FOM"],
                    dtype=["str", "int", "str", "str", "float", "float", "float", "float", "float", "str", "float", "float", "float", "float", "float", "float", "float", "float"],
)
for tic_session in tic_sessions:
    id = tic_session["TIC ID"]
    t = tic_session["MidTime JD"]
    for member in ["A", "B", "C"]:
        ephem = ephemerides[(ephemerides["Local ID"] == id) & (ephemerides["Member"] == member)]
        if len(ephem) == 0:
            continue
        ephem = ephem[0]
        period = ephem["Period"]
        for comp_letter, component in {"a": "Primary", "b": "Secondary"}.items():
            if ephem[f"T0 {component}"] > 0 and ephem[f"Duration {component}"] > 0:
                t0 = float(ephem[f"T0 {component}"]) + 2457000
                half_duration = ephem[f"Duration {component}"] / 24 / 2
                depth_name = f"Depth {component}"
                if depth_text := ephem[depth_name]:
                    depth = int(depth_text) / 1000 # kostov gives depth as ppt
                else:
                    depth = 0
                cycles = round((t - t0) / period)
                eclipse_beg = t0 + (period * cycles) - half_duration
                eclipse_end = eclipse_beg + 2 * half_duration
                confirmed = eclipse_beg < t < eclipse_end
                if confirmed:
                    mid_eclipse = (eclipse_beg + eclipse_end) / 2
                    eclipse_fraction = abs(t - mid_eclipse) / half_duration
                    fom = depth * (1 - eclipse_fraction) # this assumes linear slope of light curve, better would be gaussian
                    eclipses.add_row([id, tic_session["Speckle Session"], member, comp_letter, eclipse_beg, eclipse_end, tic_session["StartTime JD"], t, tic_session["EndTime JD"], tic_session["MidTime UTC"], tic_session["MidTime Byear"], t0, period, half_duration, depth, (t - t0) / period, eclipse_fraction, fom])
print(len(eclipses), len(tic_sessions))

eclipses.sort("FOM", reverse=True)
eclipses.write("tic eclipses.csv", overwrite=True)

list(eclipses["TIC ID"])

# foo = eclipses[["TIC ID", "Member", "Component", "Session Mid JD", "Period", "Half Duration Days", "Depth", "Eclipse Fraction", "FOM"]]
# foo#[foo["TIC ID"] == "TIC 286470992"]

34 161


['TIC 269811101',
 'TIC 125952257',
 'TIC 278465736',
 'TIC 443862276',
 'TIC 251757935',
 'TIC 367448265',
 'TIC 178953404',
 'TIC 125952257',
 'TIC 470710327',
 'TIC 285681367',
 'TIC 470710327',
 'TIC 317863971',
 'TIC 130276377',
 'TIC 286470992',
 'TIC 344541836',
 'TIC 344541836',
 'TIC 89278612',
 'TIC 168789840',
 'TIC 344541836',
 'TIC 283940788',
 'TIC 470710327',
 'TIC 336882813',
 'TIC 278465736',
 'TIC 470710327',
 'TIC 161043618',
 'TIC 317863971',
 'TIC 161043618',
 'TIC 284814380',
 'TIC 283940788',
 'TIC 407060024',
 'TIC 283940788',
 'TIC 375325607',
 'TIC 278352276',
 'TIC 327885074']

In [45]:
table_1 = tb.Table.read("Eclipses from Table 1.csv")

table_1["JD"] = Time(table_1["Byear"], format="byear").jd

table_1


TIC ID,Mag,Byear,Chisq,Seeing,Theta,Rho,Delta m,Lambda,Delta lambda,Notes,Eclipse,JD
str13,float64,float64,float64,float64,float64,float64,str5,int32,int32,str70,str2,float64
TIC 25818450,11.45,2022.7447,6.19,0.93,181.0,0.0633,1.51,692,40,0.06\\,--,2459851.8576367144
TIC 25818450,11.45,2022.7447,0.67,1.22,181.2,0.0639,1.33,880,50,0.08\\,--,2459851.8576367144
TIC 45160946,13.7,2023.9106,--,--,163.1,0.1869,1.8,824,170,$\cdots$ \textcolor{red}{Noisy} \\,--,2460277.693516273
TIC 75740921,13.22,2023.9106,--,--,95.9,0.3986,0.4,824,170,$\cdots$ \textcolor{red}{Noisy} \\,--,2460277.693516273
TIC 78568780,11.51,2022.7479,3.79,0.96,246.9,0.2632,1.26,692,40,0.25 \textcolor{violet}{Aa Eclipse (4\%)}\\,Aa,2459853.0264117504
TIC 78568780,11.51,2022.7479,0.19,0.89,247.0,0.2631,1.2,880,50,0.23\\,--,2459853.0264117504
TIC 89278612,11.12,2022.7443,3.41,0.82,184.7,0.0894,0.61,692,40,0.07\\,--,2459851.7115398347
TIC 89278612,11.12,2022.7443,0.12,0.83,186.2,0.0894,0.55,880,50,0.07\\,--,2459851.7115398347
TIC 97356407,6.31,2022.8655,93.14,1.22,228.9,0.1103,4.33,692,40,0.13\\,--,2459895.978894327


## Now, do a loose join to Table 1 to look for agreement

In [68]:
eclipses["Byear"] = eclipses["Session Mid Byear"]
tb.join(table_1, eclipses, keys=["TIC ID"], join_funcs={"TIC ID": lambda x, y: x==y}, join_type="outer")#, "Byear": lambda x, y: abs(x - y) < 0.0001})

ValueError: operands could not be broadcast together with shapes (95,) (34,) 

In [108]:
# Example tables
table1 = tb.Table({'id': [1, 2, 3, 4],
                'value': [0.1, 0.2, 0.3, 0.4]})
table2 = tb.Table({'id': [3, 6, 2, 1],
                'value': [0.100001, 0.300001, 20, .1]})
# table2 = tb.Table({'id': [3, 6],
#                 'value': [0.100001, 0.300001]})

# Custom join function for floating-point comparison with tolerance
def float_approx_equal(col1, col2, tol=1e-5):
    ids1, ids2 = [], []
    for i, val1 in enumerate(col1):
        for j, val2 in enumerate(col2):
            if abs(col1 - col2) < tol:
                ids1.append(i)
                ids2.append(j)
    return ids1, ids2

# Join tables with custom floating-point comparison
# joined_table = tb.join(table1, table2, keys='value', join_type='outer')
joined_table = tb.join(table1, table2, join_funcs={'value': float_approx_equal}, join_type='outer')

print(joined_table)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [106]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

sc1 = SkyCoord([0, 1, 1.1, 2], [0, 0, 0, 0], unit='deg')
sc2 = SkyCoord([0.5, 1.05, 2.1], [0, 0, 0], unit='deg')

tol = 0.1 * u.deg
join_func = join_skycoord(tol)
sc1, sc2, join_func(sc1, sc2)  # Associate each coordinate with unique source ID

t1 = Table([sc1], names=['sc'])
t2 = Table([sc2], names=['sc'])
t12 = table.join(t1, t2, join_funcs={'sc': join_skycoord(tol)})
print(t12)
print(join_func(sc1, sc2))  # Note new `sc_id` column with the IDs from join_func()
t1, t2

sc_id   sc_1    sc_2  
      deg,deg deg,deg 
----- ------- --------
    1 1.0,0.0 1.05,0.0
    1 1.1,0.0 1.05,0.0
(array([2, 1, 1, 3]), array([4, 1, 5]))


(<Table length=4>
    sc   
 deg,deg 
 SkyCoord
 --------
  0.0,0.0
  1.0,0.0
  1.1,0.0
  2.0,0.0,
 <Table length=3>
    sc   
 deg,deg 
 SkyCoord
 --------
  0.5,0.0
 1.05,0.0
  2.1,0.0)